# Example Notebook for Network Visualization

### Change working directory to this root

In [1]:
import os
os.chdir("..")

### Imports

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import hydra

from models.modules.base import load_inner_model_state_dict
from models.tokenizers.char import CharLevelTokenizer
from models.modules.diffusion_llm import DiffusionLLMLightningModule

### Load model

In [3]:
# Infer device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current device is {device}")

# Load network
with hydra.initialize(version_base=None, config_path="../config"):
    cfg = hydra.compose(config_name="dllm")
cfg.learning.batch_size = 128
weights_path = "weights/dllm.ckpt"
model: DiffusionLLMLightningModule = hydra.utils.instantiate(cfg.module)
model = load_inner_model_state_dict(model, weights_path).to(device).eval()

Current device is cuda


/home/vovatch/code/my/trm_sequence/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


### Load data loader

In [4]:
data_module = hydra.utils.instantiate(cfg.data)
data_module.setup("val")
loader = data_module.val_dataloader()

2025-10-14 21:55:08 - INFO - Loaded mappings from data/tiny_shakespeare/meta.pkl, vocabulary size 65
2025-10-14 21:55:08 - INFO - Loaded tokenized data from data/tiny_shakespeare/train.bin
2025-10-14 21:55:08 - INFO - Loaded mappings from data/tiny_shakespeare/meta.pkl, vocabulary size 65
2025-10-14 21:55:08 - INFO - Loaded tokenized data from data/tiny_shakespeare/val.bin
2025-10-14 21:55:08 - INFO - Loaded mappings from data/tiny_shakespeare/meta.pkl, vocabulary size 65
2025-10-14 21:55:08 - INFO - Loaded tokenized data from data/tiny_shakespeare/test.bin


### Load Tokenizer

In [5]:
tokenizer = CharLevelTokenizer()

2025-10-14 21:55:08 - INFO - Loaded mappings from data/tiny_shakespeare/meta.pkl, vocabulary size 65


## Inference

### Perform inference

In [7]:
TEXT = "What is the meaning of life?"
tokenized_text = tokenizer.encode(TEXT)
tokenized_text = torch.tensor(tokenized_text).unsqueeze(0)

with torch.no_grad():
    generated_tokens = model.generate(init_tokens=tokenized_text, seq_len=1024, vocab_size=66)
generated_text = tokenizer.decode(generated_tokens[0, ...].cpu().numpy())

print(generated_text)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
